# Transformers

Autoregressive large language models are build on top of transformers architecture.

Transformer is a neural network architecture based on self-attention mechanisms, avoiding thus recurrence 
connections. This way sequence problems can be solved in parallel, making training large models faster.

Self attention mechanisms relies on compute similarity between embeddings. For NLP they can create relations between words helps a given task such as translation.

I think self attention mechanisms works as learning to read without knowing grammar. For example, "The boy kicked the ball and it disappered." Attention layer can process "it" while at same time
is looking for other words. With time, I believe the layer will start to perceive that when objects appears in a sentece "it" will be appearing to replace or determine the object itself. This is what a pronoum does. If this happens without be parallel, it will be very dificult to perceive this relations.

To understand in detail the attention mechanisms The Attention is All You Need paper [2] is a good place to
understand the transformer architecture.

I create my personal finds and placed on this repositoy https://github.com/ramonlins/obsidian/tree/master/Papers/transformers/Ashish%20Vaswani.

To access the material is necessary to install obisidian and excalidraw plugin.

In attempt to create a simple example to implement transforms from scratch I created a simple 
overview of the architecture abstracting some parts such as residual connections, stack of layers and multiple heads.

![transformer](./images/transformer.png "Figure 1")

The idea here is to create a squence model to predict the next character vowel based on a sequence of three characters extracted from the portugues alphabet. 


In this simpler ilustration the encoder is composed by a attention layer connected with a feed forward network; 
Its outputs are connected with another attention layer togeter with the output of the masked attention layer; This attention layer output is connected to another feedforward layer that pass through a linear and
softmax operation.

The input sequence is first embedded into word vectors using an embedding layer. These word vectors serve as inputs for both query and key transformations.

Separate linear transformations with learned weight matrices are then applied to these word embeddings to obtain query vectors (Q) and key vectors (K). The purpose of these separate transformations is to project each word's embedding into lower-dimensional spaces that capture different aspects of their meaning.

SQ and K embeddings differ because they undergo distinct linear transformations with unique weight matrices during computation.

In the Transformer model, during training, the Q (query) matrix learns to search for relevant information based on what K (keys) are showing as reference.

The similarities are highlighted passing trough softmax working like amplifiers to the synaptic connections for a given input and task in matrix V (the data itself).


## Finding next vowel
pt-alphabet is defined by:

"abcdefghijklmnopqrstuvwxyz"

Dataset

In [98]:
x = "-abcdefghijklmnopqrstuvwxyz#"
emb = {}
pos = {}
for i, c in enumerate(x):
    pos[c] = i         # position i mapping
    #emb[c] = format(i,'05b') # binary encoding mapping        

In [99]:
len(x)

28

In [100]:
# Create all possible combinations of input and ouputs
combinations = "---abcdefghijklmnopqrstuvwxyz#"
vowels = 'aeoiu#'
n_seq = 3
n = len(x)

vowel_index = {}
for i, v in enumerate(combinations):
    if v in vowels:
        vowel_index[v] = i
        
samples = []
dim_alphapet = len(combinations)
for i in range(dim_alphapet-2):
    input_seq = []
    for j in range(n_seq):
       input_seq.append(pos[combinations[i+j]])
    #input_seq = combinations[i:i+3] 
    for vowel, idx in vowel_index.items():
        if idx > i+2:
            #next_vowel = vowel
            next_vowel = pos[vowel]
            break
        
        if i+2 >= 23:
            next_vowel = pos['#']
            break
        
    samples.append([input_seq, next_vowel])
    

Embeddings

In [101]:
import torch
import random

class Encoder:
    def __init__(self, n, dm, h):
        self.h = h                  # num of attention heads
        self.dm = dm                # model dim
        self.dk = dm//h
        self.dv = 2                 # model dim over num of attention heads
        
        self.embedding_layer = torch.nn.Embedding(n, dm)  # n x dm
        
        self.wQ = torch.nn.Linear(dm, self.dk)  # dm x dk
        self.wK = torch.nn.Linear(dm, self.dk)  # dm x dk
        self.wV = torch.nn.Linear(dm, self.dv)  # dm x dv
        self.ff1 = torch.nn.Linear(self.dv, dm*4)
        self.relu = torch.nn.ReLU()
        self.ff2 = torch.nn.Linear(dm*4, dm)
        self.fc = torch.nn.Linear(3*dm, 6)
        #self.ff3 = torch.nn.Linear(1, 6)
        
    def embedding(self, x):
        return self.embedding_layer(x) #(b x t x n) x (n x dm) -> (b x t x dm)
       
    def attention(self, x_seq):
        # Word embedding
        pos_emb = self.embedding(x_seq)
        
        # Search
        Q = self.wQ(pos_emb)  # (b x t x dm) x (dm x dk) -> (b x t x dk)
        # Reference
        K = self.wK(pos_emb)  # (b x t x dm) x (dm x dk) -> (b x t x dk)
        # Attention features
        V = self.wV(pos_emb)  # (b x t x dm) x (dm x dv) -> (b x t x dv)
        
        # Similarity
        QK = torch.matmul(Q, K.T)  # (t x dk) x (dk x t) -> (t x t) *removing batch info
        QKn = QK / torch.sqrt(torch.tensor([self.dm]))
        
        # Highlight similar words
        P = torch.nn.Softmax(dim=0)(QKn)    # (t x t)
        
        # Focus on specific features
        H = torch.matmul(P, V)              # (t x t) x (t x dv) -> (t x dv)
        
        # Attention latent space
        R = self.ff1(H)  # (t x dv) x (dv x dm*4) -> (t x dm*4)
        A = self.ff2(R)  # (t x dm*4) x (dm*4 x dm) -> (t x dm)
        
        # Probabilities
        flatten_a = A.view(-1)                  # 1 x (t.dm))
        logits = self.fc(flatten_a)             # t.dm x num of labels)
        p = torch.nn.Softmax(dim=0)(logits)     # prob of each label
        
        return p
    
# class Decoder(Encoder):
#     def __init__(self, n, dm, h):
#         super().__init__(n, dm, h)
        
#     def attention(self):
#         pass
        
#     def mask_attention():
#         pass

Build

In [102]:
import random

class transformer:
    def __init__(self, n, dm, h):
        self.y_seq = pos[random.choice(vowels)]
        
        self.encoder = Encoder(n=n, dm=dm, h=h)
        #self.decoder = Decoder()
        
    def forward(self, x):
        a = self.encoder.attention(x)
        
        #de_e = self.decoder.embedding(self.y_seq)
           
        #h = self.decoder.mask_attention(self.y_seq)
        
        #p   = self.decoder.attention(a, h)
            
        #self.y_seq = pos[vowels[max(enumerate(p), key=lambda x: x[1])]]
                
        return a

In [103]:
transformer = transformer(n=n, dm=16, h=1)

In [104]:
data = samples[0]
x_seq = torch.tensor(data[0])
x_seq

tensor([0, 0, 0])

In [105]:
y = transformer.forward(x_seq)

In [106]:
y

tensor([0.1180, 0.1877, 0.2365, 0.1751, 0.1524, 0.1302],
       grad_fn=<SoftmaxBackward0>)

In [107]:
vowels[max(enumerate(y), key=lambda x: x[1])[0]]

'o'

In [ ]:
## Training only with attention layer